## Preparing the imput - circuit implementation of a query gate

f_1: 0 -> 0     f_2: 0 -> 0     f_3: 0 -> 1     f_4: 0 -> 1
     1 -> 0          1 -> 1          1 -> 0          1 -> 1

In [3]:
from qiskit import QuantumCircuit

def deutsch_function(case: int):
    #generate valid deutsch function as a QuantumCircuit

    if case not in [1, 2, 3, 4]:
        raise ValueError("'case' must me 1, 2, 3 or 4.")

    f = QuantumCircuit(2)
    if case in [2, 3]:
        f.cx(0, 1)
    if case in [3, 4]:
        f.x(1)
    return f


In [4]:
display(deutsch_function(1).draw())
display(deutsch_function(2).draw())
display(deutsch_function(3).draw())
display(deutsch_function(4).draw())

q_0: 
     
q_1:

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

### Deutsch_function (Uf) será implementada após a porta hadamard, seu efeito fará com que o estado anterior se transforme de acordo com a tabela
### lembrando que para o quadradinho, eh o bit de controle e o X eh o qubit q sera flipado caso o qubit de controle for 1

Para $f_1(x) = 0$ $\newline$
teremos:  $U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus 0} = \ket{x}\ket{y}$

Para $f_2(x) = x$ $\newline$
teremos:  $U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus x}$

Para $f_3(x) = 1 \oplus x$ $\newline$ teremos:  $U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus (1 \oplus x)}$

Para $f_4(x) = 1$ $\newline$ teremos:  $U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus 1}$

Ao aplicar de novo uma porta hadamard no Qubit de cima, temos que, para
- $f_1$ teremos $\ket{0}$ (constante)
- $f_2$ teremos $\ket{1}$ (balanceada)
- $f_3$ teremos $\ket{1}$ (balanceada)
- $f_4$ teremos $\ket{0}$ (constante)

## Implementando o circuito de deutsch

In [5]:
def deutsch_circuit(function: QuantumCircuit):
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n) #o segundo argumento é o numero de bits classicos que terá armazenado o resultado da medida.

    #se n = 1, entao, qc.x(1) aplicará uma porta CNOT no segundo qubit pois para o primeiro seria qc.x(0), já range(2) aplicará uma porta H no 2 primeiros qubits (0) e (1)
    qc.x(n)
    qc.h(range(n+1))

    qc.barrier()
    qc.compose(function, inplace=True) #aplica o circuito oracular, inplace = true faz com que a funcao oracular seja implementada diretamente em qc ao inves de retornar um novo circuito
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n), range(n)) #mede os qubits de entrada e armazena os resultados nos bits clássicos correspondentes.

    return qc

In [6]:
display(deutsch_circuit(deutsch_function(3)).draw())

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

## Por fim, criando uma nova função que recebe $f_n, n = 1, 2, 3, 4$ e retorna "balanceada" ou "constante"

#### Usando um simulador clássico

In [7]:
from qiskit_aer import AerSimulator

def deutsch_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch function is constant or balanced.
    """
    qc = deutsch_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if measurements[0] == "0":
        return "constant"
    return "balanced"

In [8]:
f = deutsch_function(4)
display(deutsch_algorithm(f))

'constant'